In [28]:
%pip install opendatasets


In [29]:
!unzip "/content/Mask_wearing_dataset.zip" -d output

Archive:  /content/Mask_wearing_dataset.zip
   creating: output/Mask wearing dataset/
   creating: output/Mask wearing dataset/train/
   creating: output/Mask wearing dataset/train/images/
  inflating: output/Mask wearing dataset/train/images/-1x-1_jpg.rf.69d9b61e3cdb8a9047dad25099fcc8ef.jpg  
  inflating: output/Mask wearing dataset/train/images/-I1-MS09uaqsLdGTFkgnS0Rcg1mmPyAj95ySg_eckoM_jpeg.rf.b60002517c9371bba207314a49452cf1.jpg  
  inflating: output/Mask wearing dataset/train/images/0002526673_jpg.rf.a13de6faa0e2e5250d913ccf27b4c769.jpg  
  inflating: output/Mask wearing dataset/train/images/000b7b75-1600_jpg.rf.35b739ea6899972b07aab7225d341307.jpg  
  inflating: output/Mask wearing dataset/train/images/000_1ov3n5_0_jpeg.rf.edad89426b118f4c76ba4ae3790b3f31.jpg  
  inflating: output/Mask wearing dataset/train/images/001_1024_jpeg.rf.82de51e9c6db87b2941a34e6c65de14a.jpg  
  inflating: output/Mask wearing dataset/train/images/003_1024_jpeg.rf.3d3235de8495eb432dc3c54d17287980.jpg  
 

In [30]:
with open("/content/data.yaml", "w") as f:
    f.write("""
path: /content/output/Mask_wearing_dataset

train: train/images
val: valid/images

nc: 2
names: ['mask', 'no-mask']
    """)

In [31]:
%pip install ultralytics

In [32]:
from ultralytics import YOLO

In [33]:
model = YOLO("yolo11n")

In [36]:
model.train(
    data="/content/data.yaml",
    epochs=100,          # Extended training
    imgsz=640,         # High resolution for small objects
    batch=4,            # Batch size per GPU
    device=[0],       # Using both GPUs
    workers=4,          # Reduced workers to prevent DDP issues
    optimizer="AdamW",   # AdamW works well with YOLOv11

    # Fine-tuning learning parameters
    lr0=0.0001,         # Lower initial learning rate for fine-tuning
    lrf=0.00001,        # Minimum learning rate
    warmup_epochs=2,    # Short warmup as we're continuing
    cos_lr=True,        # Cosine schedule

    box=8.0,
    cls=4.0,
    dfl=3.0,
    single_cls = False,

    # Augmentation adjustments
    degrees=90,         # Less rotation as seeds have preferred orientations
    mosaic=0.8,        # Slightly reduced mosaic

    translate=0.2,
    scale=0.4,
    shear=15.0,        # Increased shear
    perspective=0.001,  # Minimal perspective for dense scenes
    flipud=0.5,
    fliplr=0.5,
    mixup=0.35,        # Increased mixup for blur-like effect
    copy_paste=0.4,    # Aggressive copy-paste

    # HSV augmentations
    hsv_h=0.015,       # Hue
    hsv_s=0.9,         # Increased saturation
    hsv_v=0.5,         # Increased brightness variation

    # Dense detection optimizations
    overlap_mask=True,
    mask_ratio=4,      # Integer value for mask downsampling
    nms=True,

    # Training stability
    close_mosaic=25,   # Disable mosaic earlier


    exist_ok=True,
    resume=False       # Start new training with last weights
)

Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/data.yaml, epochs=100, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=4, project=None, name=train, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=25, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, lin

100%|██████████| 755k/755k [00:00<00:00, 17.5MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/output/Mask_wearing_dataset/train/labels... 120 images, 0 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<00:00, 1752.24it/s]

train: New cache created: /content/output/Mask_wearing_dataset/train/labels.cache


...

                   all         29        162      0.614      0.604      0.588       0.22



100 epochs completed in 0.167 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.17it/s]


                   all         29        162      0.671      0.574      0.601      0.241
                  mask         28        142      0.816      0.697      0.823      0.328
               no-mask          9         20      0.526       0.45      0.378      0.153
Speed: 0.2ms preprocess, 13.4ms inference, 0.0ms loss, 30.9ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ed21a81e610>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [37]:
trained_model = YOLO("/content/runs/detect/train/weights/last.pt")

In [39]:

trained_model.train(
    data="/content/data.yaml",
    epochs=200,           # Shorter fine-tuning period
    imgsz=640,
    batch=16,            # Can use larger batch for fine-tuning
    device=[0],
    optimizer="AdamW",

    # Fine-tuning specific parameters
    lr0=0.0001,          # Lower learning rate for fine-tuning
    lrf=0.00001,
    warmup_epochs=1,     # Shorter warmup

    # Reduced augmentation for fine-tuning
    mosaic=0.5,
    mixup=0.2,

    exist_ok=True,
    resume=False
)

Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/runs/detect/train/weights/last.pt, data=/content/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

train: Scanning /content/output/Mask_wearing_dataset/train/labels.cache... 120 images, 0 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/output/Mask_wearing_dataset/valid/labels.cache... 29 images, 0 backgrounds, 0 corrupt: 100%|██████████| 29/29 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: AdamW(lr=0.0001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         29        162       0.72       0.79      0.816      0.537



200 epochs completed in 0.193 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.68it/s]


                   all         29        162      0.811      0.811       0.87      0.583
                  mask         28        142      0.916      0.782      0.924      0.632
               no-mask          9         20      0.706      0.841      0.817      0.534
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ed22f5784d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
!rm -rf "/content/output"

In [41]:
!zip -r "output.zip" "/content/runs/detect/train"

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [42]:
import locale
import os

# Set the locale to UTF-8
os.environ['LC_ALL'] = 'C.UTF-8'
os.environ['LANG'] = 'C.UTF-8'

In [43]:
!zip -r "output.zip" "/content/runs/detect/train"

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [44]:
import zipfile
import os

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join(path, '..')))

with zipfile.ZipFile('output.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir('/content/runs/detect/train', zipf)

In [7]:
from ultralytics import YOLO
trained_model = YOLO("./weights/best.pt")
results = trained_model('./Mask wearing dataset/valid/images/10fea_8011463765e2aa28b386cf_jpg.rf.7981dc5de795ab4252631683e0de84ae.jpg')[0]
results.show()


image 1/1 c:\programming files\Python\Siet\Mask wearing dataset\valid\images\10fea_8011463765e2aa28b386cf_jpg.rf.7981dc5de795ab4252631683e0de84ae.jpg: 384x640 2 masks, 1 no-mask, 168.3ms
Speed: 10.1ms preprocess, 168.3ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
